## Protobuf

Idea: 
Protobuf descriptors → schema graph (files, messages, fields, oneofs, imports).

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag


In [ ]:
import google.protobuf.descriptor_pb2 as descriptor_pb2
from google.protobuf import text_format
from google.protobuf.descriptor import FileDescriptor, Descriptor, FieldDescriptor
import tempfile
import os

# Create a sample .proto file content
proto_content = """
syntax = "proto3";

package example;

message Person {
  string name = 1;
  int32 id = 2;
  string email = 3;
  repeated PhoneNumber phones = 4;
  
  message PhoneNumber {
    string number = 1;
    PhoneType type = 2;
  }
  
  enum PhoneType {
    MOBILE = 0;
    HOME = 1;
    WORK = 2;
  }
}

message AddressBook {
  repeated Person people = 1;
}
"""

print("Sample protobuf schema created")
print(proto_content)

In [ ]:
# Define data structures to represent protobuf schema elements
from dataclasses import dataclass
from typing import List, Optional, Dict, Any

@dataclass
class ProtoField:
    name: str
    type: str
    number: int
    label: str = "optional"  # optional, required, repeated
    
@dataclass  
class ProtoEnum:
    name: str
    values: Dict[str, int]
    
@dataclass
class ProtoMessage:
    name: str
    fields: List[ProtoField]
    nested_messages: List['ProtoMessage']
    enums: List[ProtoEnum]
    
@dataclass
class ProtoFile:
    package: str
    messages: List[ProtoMessage]
    imports: List[str]
    syntax: str = "proto3"

In [ ]:
# Add spytial annotations to our protobuf data structures for better visualization

@orientation(selector='fields', directions=['below'])
@orientation(selector='nested_messages', directions=['right'])
@orientation(selector='enums', directions=['right'])
@atomColor(selector='self', value='lightblue')
@dataclass
class AnnotatedProtoMessage:
    name: str
    fields: List[ProtoField]
    nested_messages: List['AnnotatedProtoMessage']
    enums: List[ProtoEnum]

@atomColor(selector='self', value='lightgreen')
@dataclass
class AnnotatedProtoField:
    name: str
    type: str
    number: int
    label: str = "optional"

@atomColor(selector='self', value='lightyellow')
@dataclass
class AnnotatedProtoEnum:
    name: str
    values: Dict[str, int]

@orientation(selector='messages', directions=['below'])
@atomColor(selector='self', value='lightcoral')
@dataclass
class AnnotatedProtoFile:
    package: str
    messages: List[AnnotatedProtoMessage]
    imports: List[str]
    syntax: str = "proto3"

In [ ]:
def parse_protobuf_schema(proto_content: str) -> AnnotatedProtoFile:
    """Parse a protobuf schema string and convert it to our visualization format."""
    
    # For this demo, we'll manually parse the simple schema
    # In a real implementation, you'd use protoc or protobuf reflection
    
    # Create enum
    phone_type_enum = AnnotatedProtoEnum(
        name="PhoneType",
        values={"MOBILE": 0, "HOME": 1, "WORK": 2}
    )
    
    # Create fields for PhoneNumber message
    phone_number_fields = [
        AnnotatedProtoField(name="number", type="string", number=1, label="optional"),
        AnnotatedProtoField(name="type", type="PhoneType", number=2, label="optional")
    ]
    
    # Create nested PhoneNumber message
    phone_number_message = AnnotatedProtoMessage(
        name="PhoneNumber",
        fields=phone_number_fields,
        nested_messages=[],
        enums=[]
    )
    
    # Create fields for Person message
    person_fields = [
        AnnotatedProtoField(name="name", type="string", number=1, label="optional"),
        AnnotatedProtoField(name="id", type="int32", number=2, label="optional"),
        AnnotatedProtoField(name="email", type="string", number=3, label="optional"),
        AnnotatedProtoField(name="phones", type="PhoneNumber", number=4, label="repeated")
    ]
    
    # Create Person message
    person_message = AnnotatedProtoMessage(
        name="Person",
        fields=person_fields,
        nested_messages=[phone_number_message],
        enums=[phone_type_enum]
    )
    
    # Create AddressBook message
    address_book_fields = [
        AnnotatedProtoField(name="people", type="Person", number=1, label="repeated")
    ]
    
    address_book_message = AnnotatedProtoMessage(
        name="AddressBook",
        fields=address_book_fields,
        nested_messages=[],
        enums=[]
    )
    
    # Create the file structure
    proto_file = AnnotatedProtoFile(
        package="example",
        messages=[person_message, address_book_message],
        imports=[],
        syntax="proto3"
    )
    
    return proto_file

# Parse our example schema
schema = parse_protobuf_schema(proto_content)
print("Parsed protobuf schema structure:")
print(f"Package: {schema.package}")
print(f"Messages: {[msg.name for msg in schema.messages]}")
print(f"Person nested messages: {[msg.name for msg in schema.messages[0].nested_messages]}")
print(f"Person enums: {[enum.name for enum in schema.messages[0].enums]}")

In [ ]:
# Generate the spytial visualization of the protobuf schema
result = diagram(schema, method='file', auto_open=False)
print(f"Generated protobuf schema visualization: {result}")

# Let's also visualize just the Person message for a focused view
person_msg = schema.messages[0]  # Person message
person_result = diagram(person_msg, method='file', auto_open=False)
print(f"Generated Person message visualization: {person_result}")

In [ ]:
# Advanced Example: Using actual protobuf descriptors
# This demonstrates how to work with real protobuf descriptor objects

def descriptor_to_spytial(file_descriptor) -> AnnotatedProtoFile:
    """Convert a real protobuf FileDescriptor to our spytial visualization format."""
    
    def field_to_proto_field(field_desc) -> AnnotatedProtoField:
        # Map protobuf field types to string representations
        type_map = {
            1: 'double', 2: 'float', 3: 'int64', 4: 'uint64', 5: 'int32',
            6: 'fixed64', 7: 'fixed32', 8: 'bool', 9: 'string', 10: 'group',
            11: 'message', 12: 'bytes', 13: 'uint32', 14: 'enum', 15: 'sfixed32',
            16: 'sfixed64', 17: 'sint32', 18: 'sint64'
        }
        
        label_map = {1: 'optional', 2: 'required', 3: 'repeated'}
        
        field_type = type_map.get(field_desc.type, 'unknown')
        if field_desc.type in [11, 14]:  # message or enum
            field_type = field_desc.type_name.split('.')[-1] if field_desc.type_name else field_type
            
        return AnnotatedProtoField(
            name=field_desc.name,
            type=field_type,
            number=field_desc.number,
            label=label_map.get(field_desc.label, 'optional')
        )
    
    def message_to_proto_message(msg_desc) -> AnnotatedProtoMessage:
        fields = [field_to_proto_field(f) for f in msg_desc.field]
        nested_msgs = [message_to_proto_message(nested) for nested in msg_desc.nested_type]
        enums = [AnnotatedProtoEnum(
            name=enum_desc.name,
            values={val.name: val.number for val in enum_desc.value}
        ) for enum_desc in msg_desc.enum_type]
        
        return AnnotatedProtoMessage(
            name=msg_desc.name,
            fields=fields,
            nested_messages=nested_msgs,
            enums=enums
        )
    
    messages = [message_to_proto_message(msg) for msg in file_descriptor.message_type]
    
    return AnnotatedProtoFile(
        package=file_descriptor.package,
        messages=messages,
        imports=list(file_descriptor.dependency),
        syntax=file_descriptor.syntax or "proto2"
    )

print("Advanced protobuf descriptor parser created!")
print("This can be used with real .proto files compiled with protoc.")

In [ ]:
# Creating a Schema Graph Visualization
# This shows relationships between messages, fields, and types

@orientation(selector='nodes', directions=['horizontal'])
@orientation(selector='edges', directions=['below'])
@atomColor(selector='self', value='purple')
@dataclass
class SchemaGraph:
    """Represents the dependency graph of a protobuf schema"""
    nodes: List[Any]  # Messages, enums, etc.
    edges: List['SchemaEdge']  # Relationships
    
@atomColor(selector='self', value='orange')
@dataclass
class SchemaEdge:
    """Represents a relationship in the schema graph"""
    from_node: str
    to_node: str
    relationship: str  # "contains", "references", "extends", etc.

def create_schema_graph(proto_file: AnnotatedProtoFile) -> SchemaGraph:
    """Create a graph representation of the protobuf schema showing relationships."""
    
    nodes = []
    edges = []
    
    # Add the file itself as root node
    nodes.append(proto_file)
    
    for message in proto_file.messages:
        nodes.append(message)
        edges.append(SchemaEdge(proto_file.package, message.name, "contains"))
        
        # Add field relationships
        for field in message.fields:
            nodes.append(field)
            edges.append(SchemaEdge(message.name, field.name, "has_field"))
            
            # If field references another message type, add that edge
            if field.type in [msg.name for msg in proto_file.messages]:
                edges.append(SchemaEdge(field.name, field.type, "references"))
        
        # Add nested message relationships
        for nested in message.nested_messages:
            edges.append(SchemaEdge(message.name, nested.name, "nests"))
            
        # Add enum relationships
        for enum in message.enums:
            nodes.append(enum)
            edges.append(SchemaEdge(message.name, enum.name, "defines"))
    
    return SchemaGraph(nodes=nodes, edges=edges)

# Create and visualize the schema graph
schema_graph = create_schema_graph(schema)
graph_result = diagram(schema_graph, method='file', auto_open=False)
print(f"Generated schema graph visualization: {graph_result}")
print(f"Graph has {len(schema_graph.nodes)} nodes and {len(schema_graph.edges)} edges")

## Protobuf Schema Visualizer

This notebook demonstrates how to create a comprehensive visualizer for protobuf descriptors using spytial. The visualizer includes:

### Key Features:

1. **Message Visualization**: Shows protobuf messages with their fields, nested messages, and enums
2. **Schema Graph**: Displays relationships between different protobuf elements 
3. **Spatial Annotations**: Uses spytial decorators to organize the layout:
   - Messages are colored light blue
   - Fields are colored light green  
   - Enums are colored light yellow
   - Schema graphs use purple and orange for nodes and edges

### Visualization Types:

- **Individual Messages**: Focus on a single message structure
- **Complete Schema**: View the entire protobuf file structure
- **Dependency Graph**: Show relationships and references between schema elements

The visualizer handles:
- Field types and labels (optional, required, repeated)
- Nested messages and enums
- Cross-references between message types
- Package organization

### Usage:

1. Parse protobuf content into annotated data structures
2. Apply spytial annotations for better layout
3. Generate interactive HTML visualizations
4. Create dependency graphs showing schema relationships